In [ ]:
import matplotlib.pyplot as plt, pandas as pd, requests, time
from IPython.display import clear_output, display
import requests, pandas as pd, pprint, time
BASE = "http://localhost:8000"
from IPython.display import Image, HTML
%matplotlib inline
import matplotlib.pyplot as plt, matplotlib.dates as mdates
import pandas as pd, requests, time
from IPython.display import clear_output, display

In [ ]:
print(requests.get("http://localhost:8000/trades/plain?limit=5").text)

In [ ]:
pprint.pp(requests.get(f"{BASE}/stats").json())

trades = pd.DataFrame(requests.get(f"{BASE}/profit").json())
display(trades.tail())

In [ ]:
requests.get(f"{BASE}/profit/total/plain").text.strip()

In [ ]:
bars      = pd.DataFrame(requests.get(f"{BASE}/bars", params={"limit": 600}).json())
decisions = pd.DataFrame(requests.get(f"{BASE}/decisions").json())


N      = 3200           # bars in window
PAUSE  = 1.0           # seconds between redraws

while True:
    bars_json  = requests.get(f"{BASE}/bars", params={"limit": N}).json()
    dec_json   = requests.get(f"{BASE}/decisions", params={"limit": 200}).json()
    bars       = pd.DataFrame(bars_json)
    decs       = pd.DataFrame(dec_json)

    if bars.empty:
        time.sleep(PAUSE)
        continue

    fig, ax = plt.subplots(figsize=(9, 4))
    t  = pd.to_datetime(bars.iso_ts)
    px = bars.close
    ax.plot(t, px, lw=1, label="close")

    window_start = bars.bucket.min()
    in_window = decs[decs.timestamp >= window_start]

    buys = in_window[in_window.decision == "BUY"]
    if not buys.empty:
        ax.scatter(pd.to_datetime(buys.timestamp, unit="s"),
                   buys.price,
                   marker="^", color="green", zorder=3, label="BUY")

    sells = in_window[in_window.decision == "SELL"]
    if not sells.empty:
        ax.scatter(pd.to_datetime(sells.timestamp, unit="s"),
                   sells.price,
                   marker="v", color="red", zorder=3, label="SELL")

    ax.set_xlim(t.iloc[0], t.iloc[-1])
    ax.set_ylim(px.min()*0.999, px.max()*1.001)
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    fig.autofmt_xdate()
    ax.legend()

    clear_output(wait=True)
    display(fig)
    plt.close(fig)
    time.sleep(PAUSE)
